In [ ]:
%pip install requests

In [14]:
import requests
path = "/api/events/22221"
base_url = "http://localhost:8002"

endpoint = f"{base_url}{path}"


In [15]:
response = requests.get(endpoint)

print(response.json())


{'id': 22221}
